# Distributed Quantum Neural Networks on Distributed Photonic Quantum Computing (DQNN) with MerLin

This implementation is based on prior work by [Chen *et al.*](https://arxiv.org/pdf/2505.08474v1) and their [repository](https://github.com/Louisanity/PhotonicQuantumTrain).

This notebook will demonstrate the Quantum Train framework with:
- A brief introduction to the concepts.
- A tutorial on how to run a simple classification task with the MNIST dataset.



## 1. Introduction


Parler de MPS, LE PROCESS

The Photonic Quantum Train algorithm leverages the efficient expressivity of photons to represent an exponentionnaly large number of photons. Indeed in the example that will follow, only 17 photons are necessary to generate the 6690 parameters used in a complete CNN model. The full pipeline can be represented by this figure.

![](images/md1.png)
Source: K.-C. Chen, C.-Y. Liu, Y. Shang, F. Burt, and K. K. Leung, “Distributed Quantum Neural Networks on Distributed Photonic Quantum Computing,” May 13, 2025, arXiv: arXiv:2505.08474. doi: 10.48550/arXiv.2505.08474.


We can observe here that 2 boson sampler are used to generate probabilities of a 13 photon system. This probability distribution of $2^13$ elements is then processed by a quantum layer represented by an MPS. The MPS has 14 sites and an output dimension of 1. The role of this layer is to take the distribution and then process it into a list of classical parameters to assign to the CNN model on the right. The user can also change the bond dimension $\chi$. Changing this hyperparameter directly controls the experessablity of the MPS layer. A higher bond dimension guves a higher expressability but also much more parameters to optimize.

NOTE: In this notebook the ADAM optimizer will be used for the boson samplers and for the MPS layer since it is faster.

## Note

The tutorial below decorticates the `run_default_exp` function in the [/papers.DQNN.lib/default_exp.py](/reproduced_papers/papers/DQNN/papers.DQNN.lib/default_exp.py) file. It is also possible to run this experiment simply by running this line. 
 >``python3 papers.DQNN.lib/runner.py --exp_to_run DEFAULT``

 Other experiments are available. See the [README](/reproduced_papers/papers/DQNN/README.md).

## 2. Setup and Imports

In [ ]:
import torch
from torch.utils.data import DataLoader
import sys
import os

sys.path.append(os.path.dirname(os.path.dirname(__file__)))
import warnings

warnings.filterwarnings("ignore")

from papers.DQNN.lib.photonic_qt_utils import (
    setup_session,
    create_boson_samplers,
    calculate_qubits,
)
from papers.DQNN.lib.model import PhotonicQuantumTrain, train_quantum_model, evaluate_model
from papers.DQNN.lib.classical_utils import create_datasets, train_classical_cnn

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

## 3. Load the data

The partial MNIST set used in the paper and experiments can be accessed easily using this function

In [ ]:
train_dataset, val_dataset, train_loader, val_loader, batch_size = create_datasets()


batch_size_qnn = 1000
train_loader_qnn = DataLoader(train_dataset, batch_size_qnn, shuffle=True)

## 4. Create the Model

In [ ]:
#Create the two boson samplers to use
bs_1, bs_2 = create_boson_samplers()

#Calculates the number of qubits needed to generate all of the parameters of the CNN.
#The nw_list_normal is a list which gives a template on how many parameters need to 
#be generated fp the classical CNN model.
n_qubit, nw_list_normal = calculate_qubits()

#Define the bond dimension of the MPS, by default we recommand 7
bond_dim=7

qt_model = PhotonicQuantumTrain(n_qubit, bond_dim=bond_dim).to(device)

# 5. Run the CNN model by itself for comparaison

To do so, the function below can be called. This function generates, trains, and prints the accuracy of the classical model. It returns it at the end for future uses. It is the same CNN structure that will be optimized by the quantum train.

In [ ]:
model = train_classical_cnn(
    train_loader,
    val_loader,
    classical_epochs=5,
)

## 6. Train the Quantum Train

To train the model the function below can be called. Here is a description of all of the parameters:

- `qt_model` : PhotonicQuantumTrain
    Model to train.
- `train_loader` : DataLoader
    Loader for standard training batches.
- `train_loader_qnn` : DataLoader
    Loader for QNN parameter training batches.
- `bs_1` : BosonSampler
    First boson sampler providing a quantum layer.
- `bs_2` : BosonSampler
    Second boson sampler providing a quantum layer.
- `n_qubit` : int
    Number of qubits used to generate the quantum states.
- `nw_list_normal` : List[float]
    Indices of network weights to keep from the generated probabilities.
- `num_training_rounds` : int
    Number of training rounds.
- `num_epochs` : int
    Number of epochs per training round for the MPS mapping network.
- `num_qnn_train_step` : int, optional
    Number of optimization steps for the QNN parameters. Default is 12. If the COBYLA optimizer is to be used, 1000 is the suggested value.
- `qu_train_with_cobyla` : bool, optional
    Whether to use COBYLA for QNN optimization. Default is False.

The updated model, the updated parameters of the boson samplers, the loss and accuracy values per training round are returned. The parameters of the model are already updated, the returned parameters are just jere for reference.

In [ ]:
qt_model,qnn_parameters, loss_list_epoch, acc_list_epoch = train_quantum_model(
        qt_model=qt_model,
        train_loader=train_loader,
        train_loader_qnn=train_loader_qnn,
        bs_1=bs_1,
        bs_2=bs_2,
        n_qubit=n_qubit,
        nw_list_normal=nw_list_normal,
        num_training_rounds=30,
        num_epochs=5,
        qu_train_with_cobyla=False,
        num_qnn_train_step=12,
    )

# 7. Use the model as you want!

A simple function who prints and returns  the performance for a test set is also available:

In [ ]:
acc,loss,gen_error=evaluate_model(
        qt_model, train_loader, val_loader, bs_1, bs_2, n_qubit, nw_list_normal
    )

## Conclusion

Other experiments are available in this repository. In order to run them and this one in just a terminal line, make sur the checkout the [README](/reproduced_papers/papers/DQNN/README.md).